## Generation-time branch attraction

This notebook focuses on (g) effects

In [1]:
# conda install ipcoal ipyrad -c conda-forge -c bioconda 

In [2]:
import numpy as np
import pandas as pd
import toytree
import toyplot, toyplot.svg
import ipcoal
import ipyrad.analysis as ipa

In [ ]:
! mkdir -p ./trees
! mkdir -p ./db

### Treestyle

In [18]:
ts = {
    'ts': 'p', 
    'layout': 'r',
    'height': 300,
    'node_sizes': 7,
    'node_style': {"stroke": "black", 'stroke-width': 2},
    'node_labels': False,
    'edge_type': 'c',
    'scalebar': False,
    'tip_labels_align': True,
    'scalebar': True,
}

### Starting simulation scenario with deep divergences

In [4]:
# get an ultrametric imbalanced tree
tree = toytree.rtree.imbtree(8, treeheight=20e6)
tree.draw(**ts);

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tc06a7e38bbaa40ecb426c660e8de338d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 260.0 300.0" width="260.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r0 r1 r2 r3 r4 r5 r6 r7 0 10000000 20000000

### Modify species tree params to make "variable g" tree

In [5]:
# set gentime on nodes
gtree = tree.set_node_values(
    feature="g", 
    values={i: 10 for i in (3,4,5,10,11,12)},
    default=1,
)

# set gentime on nodes
gtree = gtree.set_node_values(
    feature="dist", 
    values={i: j.dist / j.g for (i, j) in gtree.idx_dict.items()}
)

# draw the tree
gtree.draw(**ts);

# save to file
gtree.write("./trees/g-tree.nwk")

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t80b15fec592745798acf3e1dd181fe65" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 260.0 300.0" width="260.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r0 r1 r2 r3 r4 r5 r6 r7 0 10000000 20000000

### Simulate a concatenated sequence (chromosome)

In [10]:
# simulate a long chrom
model = ipcoal.Model(gtree, Ne=10e6, seed=123)
model.sim_loci(1, 1e6)

# save to db
model.write_loci_to_hdf5(name="g-concat", outdir="db")

# show the total number of genealogies
print('ngenealogies:', model.df.tidx.max())
print('mean len of gtree: {:.2f} bp'.format(model.df.nbps.mean()))

# show the locus (pretty high variation)
model.draw_seqview(0, 0, 50);

ngenealogies: 116401
mean len of gtree: 8.59 bp


<svg class="toyplot-canvas-Canvas" height="128.0px" id="t652e10c1a5f849ef9a7faebdc7bb3155" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 1020.0 128.0" width="1020.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r0 r1 r2 r3 r4 r5 r6 r7

### Confirm concatenation is in the anomaly zone 

Here we aim to select a scenario that will lie in the 'anomaly zone', where concatenation will yield incorrect results but a proper MSC method should infer a correct result.

In [8]:
# init raxml inference
tool = ipa.treeslider(
    data="./db/g-concat.seqs.hdf5",
    name="g-concat",
    scaffold_idxs=0,
    inference_args={
        "f": "d",
        "N": 10,
        "p": 12345,
        "x": None,
        "T": 20,
    }
)
tool.ipcluster['threads'] = 20
tool.show_inference_command()

# infer concat tree
tool.run(auto=True, force=True)

/home/deren/miniconda3/envs/ipy/bin/raxmlHPC-PTHREADS-AVX2 -f d -T 20 -m GTRGAMMA -n ... -w ... -s ... -p 12345 -N 10
building database: nwindows=1; minsnps=1
[####################] 100% 0:00:27 | inferring trees 
tree_table written to /home/deren/gentime-attraction/notebooks/analysis-treeslider/g-concat.tree_table.csv


In [9]:
# re-load from treeslider result
tw = pd.read_csv("./analysis-treeslider/g-concat.tree_table.csv")

# draw the concat tree
etree = toytree.tree(tw.tree[0]).root("r7")
etree.draw(**ts);

# save tree to file
etree.write("./trees/g-concat.nwk")

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t64f5847a792f42809b092c6c5f83a763" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 260.0 300.0" width="260.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r0 r1 r2 r3 r5 r6 r4 r7 0.0 0.3 0.6

### Simulate unlinked genealogies

In [12]:
# simulate many unlinked genealogies
model = ipcoal.Model(gtree, Ne=10e6, seed=123)
model.sim_trees(nloci=10000, nsites=1)
mtre = toytree.mtree(model.df.genealogy)

# draw example genealogies
mtre.draw();

# convert bl units to E(subst.)
for tre in mtre.treelist:
    for node in tre.idx_dict.values():
        node.dist = node.dist * 1e-8      
        
# write re-formatted trees to a tmp file
mtre.write("./trees/g-genealogies.nwk")

<svg class="toyplot-canvas-Canvas" height="200.0px" id="t3a6824e92add4a47896acc12f8cd0c66" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 560.0 200.0" width="560.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r5 r1 r0 r3 r7 r6 r4 r2 r4 r2 r0 r1 r7 r6 r5 r3 r6 r1 r3 r7 r5 r0 r4 r2 r1 r0 r7 r2 r4 r3 r6 r5

### Confirm ASTRAL-genealogies is not an anomaly


In [15]:
# infer MSC sptree with ASTRAL
ast = ipa.astral(
    data=[i.write() for i in toytree.mtree("./trees/g-genealogies.nwk")],
    name="g-astral-genealogy",
)
ast.run()

# draw sptree
atree = toytree.tree(ast.tree).root("r7")
atree.draw(**ts);

# save tree to file
atree.write("./trees/g-astral-genealogies.nwk")

[astral.5.7.1.jar]
inferred tree written to (/home/deren/gentime-attraction/notebooks/analysis-astral/g-astral-genealogy.tre)


<svg class="toyplot-canvas-Canvas" height="300.0px" id="t605e3ace8c894b00a61c8407874707a8" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 260.0 300.0" width="260.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r1 r0 r2 r3 r4 r5 r6 r7 0.0 0.9 1.8

### Simulate loci
Do 10K loci and test a few range of locus lengths to allow examining length effect.

In [14]:
# simulate many unlinked genealogies (same as above)
model = ipcoal.Model(gtree, Ne=10e6, seed=123)
model.sim_loci(nloci=10000, nsites=2e3)
model.write_loci_to_hdf5(name="g-10000l-2000s", outdir="db")
model.write_snps_to_hdf5(name="g-10000l-2000s", outdir="db")

# draw seqview
model.draw_seqview(0, 0, 50);

wrote 10000 loci to /home/deren/gentime-attraction/notebooks/db/g-10000l-2000s.seqs.hdf5
wrote 15615699 SNPs to /home/deren/gentime-attraction/notebooks/db/g-10000l-2000s.snps.hdf5


<svg class="toyplot-canvas-Canvas" height="128.0px" id="tc106c140473d43e987f21403745d11f7" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 1020.0 128.0" width="1020.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r0 r1 r2 r3 r4 r5 r6 r7

### Infer gene trees from loci


In [17]:
# init tree slider inference tool
tool = ipa.treeslider(
    data="./db/g-10000l-2000s.seqs.hdf5",
    name="g-10000l-2000s",
    scaffold_idxs=range(10000),
    inference_args={
        "N": 10,
        "f": "d",
        "p": 12345,
        "x": None,
        "T": 2,
    }
)

# infer all gene trees
tool.ipcluster['cores'] = 20
tool.ipcluster['threads'] = 2
tool.run(auto=True, force=True)

building database: nwindows=10000; minsnps=1
[####################] 100% 1:28:35 | inferring trees 
tree_table written to /home/deren/gentime-attraction/notebooks/analysis-treeslider/g-10000l-2000s.tree_table.csv


In [19]:
# load tree slider (gene tree inference) results
tw = pd.read_csv("./analysis-treeslider/g-10000l-2000s.tree_table.csv", index_col=0)

# load gene trees into toytree multitree object
mtre = toytree.mtree(tw.tree)
mtre.treelist = [i.root("r7") for i in mtre.treelist]
mtre.draw(ts='o', use_edge_lengths=False);

# write list of trees as a tmp file
mtre.write("./trees/g-genetrees.nwk")

<svg class="toyplot-canvas-Canvas" height="200.0px" id="t83626b1898b4462880277f32869efe15" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 560.0 200.0" width="560.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r0 r2 r4 r6 r3 r5 r1 r7 r3 r2 r6 r0 r1 r4 r5 r7 r0 r1 r4 r3 r5 r6 r2 r7 r0 r1 r2 r3 r5 r4 r6 r7

### Confirm SNAQ-genealogies is not an anomaly

In [52]:
# infer sptree
snaq0 = ipa.snaq(
    gtrees="/trees/g-genetrees.nwk",
    netin="./trees/g-astral-genealogies.nwk",
    name="g-snaq-genetrees-net0",
    nedges=0,
    nruns=10,
    nproc=10,
    seed=123,
)
snaq0.run()

# infer sptree
snaq1 = ipa.snaq(
    gtrees="/trees/g-genetrees.nwk",
    netin="./trees/g-astral-genealogies.nwk",
    name="g-snaq-genetrees-net1",
    nedges=1,
    nruns=10,
    nproc=10,
    seed=123,
)
snaq1.run()

using existing CF table: /home/deren/gentime-attraction/notebooks/analysis-snaq/n-snaq-genealogy-net0.CFs.csv
[SNAQ v.x.y]
[nproc = 10]
julia /home/deren/gentime-attraction/notebooks/analysis-snaq/n-snaq-genealogy-net0.jl
inferred network written to (/home/deren/gentime-attraction/notebooks/analysis-snaq/n-snaq-genealogy-net0.snaq)
[SNAQ v.x.y]
[nproc = 10]
julia /home/deren/gentime-attraction/notebooks/analysis-snaq/n-snaq-genealogy-net1.jl
inferred network written to (/home/deren/gentime-attraction/notebooks/analysis-snaq/n-snaq-genealogy-net1.snaq)


In [60]:
snaq1.admix

{'H9': (['r6'], ['r2'], 0.5, {}, '0.017')}

In [62]:
# draw tree
stree = toytree.tree(snaq1.tree).root("r7")
stree.draw(**ts, admixture_edges=snaq1.admix.values());

# write network
stree.write("trees/g-snaq-genealogies.nwk")

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t60c2f184bae842299dc146e323933e99" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 260.0 300.0" width="260.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0.017 r1 r0 r2 r3 r4 r5 r6 r7 0.0 0.9 1.8

### Estimate gene trees

Do 10K loci and test a few range of locus lengths to allow examining length effect.

In [ ]:
# simulate loci that are each 3000bp in length
model = ipcoal.Model(gtree, Ne=10e6, seed=123)
model.sim_loci(nloci=10000, nsites=2e3)
model.write_loci_to_hdf5(name="g=10000l-2000s", outdir="db")
model.write_snps_to_hdf5(name="g-10000l-2000s", outdir="db")

# draw seqview
model.draw_seqview(0, 0, 50);

In [ ]:
# init tree slider inference tool
tool = ipa.treeslider(
    data="./db/g-10000l-2000s.seqs.hdf5",
    name="g-10000l-2000s",
    scaffold_idxs=range(10000),
    inference_args={
        "N": 10,
        "f": "d",
        "p": 12345,
        "x": None,
        "T": 2,
    }
)

# infer all gene trees
tool.ipcluster['cores'] = 40
tool.ipcluster['threads'] = 2
tool.run(auto=True, force=True)

In [ ]:
# load tree slider (gene tree inference) results
tw = pd.read_csv("./analysis-treeslider/g-10000l-2000s.tree_table.csv", index_col=0)

# load gene trees into toytree multitree object
mtre = toytree.mtree(tw.tree)
mtre.treelist = [i.root("r7") for i in mtre.treelist]
mtre.draw(ts='o', use_edge_lengths=False);

# write list of trees as a tmp file
print(len(mtre), "genetrees")
mtre.write("./trees/g-genetrees.nwk")

### Infer SNAQ network from inferred gene trees

In [88]:
snaq0 = ipa.snaq(
    gtrees="./trees/g-genetrees.nwk",
    netin="./trees/g-astral-genealogies.nwk",
    name="g-genetrees-net0",
    nedges=0,
    nruns=10,
    nproc=10,
)
snaq0.run()

snaq1 = ipa.snaq(
    gtrees="./trees/g-genetrees.nwk",
    netin="./trees/g-astral-genealogies.nqk",
    name="g-genetrees-net1",
    nedges=1,
    nruns=10,
    nproc=10,
)
snaq1.run()

In [ ]:
snaq1.admix

In [ ]:
# draw tree
stree = toytree.tree(snaq1.tree).root("r7")
stree.draw(**ts, admixture_edges=snaq1.admix.values());

# save tree
stree.write("./trees/g-snaq-genetrees.nwk")

### Infer astral tree from inferred gene trees

In [20]:
# infer MSC sptree with ASTRAL
ast = ipa.astral(
    data=[i.write() for i in toytree.mtree("./trees/g-genetrees.nwk")],
    name="g-astral-genetrees",
)
ast.run()

# draw sptree
atree = toytree.tree(ast.tree).root("r7")
atree.draw(**ts);

# save tree file
atree.write("./trees/g-astral-genetrees.nwk")

[astral.5.7.1.jar]
inferred tree written to (/home/deren/gentime-attraction/notebooks/analysis-astral/g-astral-genetrees.tre)


<svg class="toyplot-canvas-Canvas" height="300.0px" id="t658b9723c10e4f0bb137482704ca7aba" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 260.0 300.0" width="260.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r1 r0 r2 r3 r4 r6 r5 r7 0.0 0.9 1.8